In [1]:
!pip install pyswarm
!pip install torchswarm
!pip install pyswarms

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import datetime as dt
import tensorflow as tf
import math
import torch
from torch import nn
from torch.nn import functional as F
from torch.autograd import Variable 
from pyswarm import pso
from pyswarms.utils.functions import single_obj as fx
from torchswarm.swarmoptimizer import SwarmOptimizer
from sklearn.metrics import f1_score, classification_report, confusion_matrix
from pyswarm import pso


from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Conv1D
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Activation
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras import layers
from tensorflow import keras
import tensorflow
from tensorflow.keras import datasets, layers, models
from tensorflow.keras.utils import to_categorical

In [3]:
df=pd.read_csv('../input/creditcardfraud/creditcard.csv')
df_fraud=df['Class']
df_select=df.drop(['Class'],axis=1)
features=np.array(df_select)
label=np.array(df_fraud)
from imblearn.over_sampling import SMOTE
from collections import Counter
from matplotlib import pyplot
from numpy import where
from sklearn.model_selection import train_test_split
oversample = SMOTE()
X_r, y = oversample.fit_resample(features, label)
# summarize the new class distribution
counter = Counter(y)
print(counter)
for label, _ in counter.items():
    row_ix = where(y == label)[0]
from sklearn.preprocessing import StandardScaler

X_r2 = StandardScaler().fit_transform(X_r)




Counter({0: 284315, 1: 284315})


In [24]:
X_r2.shape

(568630, 30)

In [4]:
X_train,X_test,y_train,y_test = train_test_split(X_r2, y, test_size=0.3)
X_train = np.reshape(X_train, (X_train.shape[0],X_train.shape[1],1))
X_test = np.reshape(X_test, (X_test.shape[0],X_test.shape[1],1))

In [5]:
X_train.shape[1]

30

In [6]:
EarlyStopper = EarlyStopping(patience=4, monitor='val_loss', mode='min')
model = Sequential()
model.add(LSTM(11,return_sequences=True, input_shape = (X_train.shape[1], 1)))
model.add(layers.Flatten())
model.add(Dense(1, kernel_initializer='lecun_uniform', activation='sigmoid'))
model.compile(loss='MeanSquaredError', optimizer='adam', metrics=["accuracy"])
model.summary()

2022-09-18 13:47:05.369493: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-09-18 13:47:05.370700: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-09-18 13:47:05.371415: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-09-18 13:47:05.372543: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compil

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 30, 11)            572       
_________________________________________________________________
flatten (Flatten)            (None, 330)               0         
_________________________________________________________________
dense (Dense)                (None, 1)                 331       
Total params: 903
Trainable params: 903
Non-trainable params: 0
_________________________________________________________________


In [48]:
def mind(m):
    l=[]
    for i in range(30):
        if m[i]==1:
            l.append(i)
    return l

In [49]:
def f_per_particle(m, alpha):
   
    total_features = X_r2.shape[1]

    
    if np.count_nonzero(m) == 0: 
         X_subset = X_r2
    else:
        l=mind(m)
        X_subset = X_r2[:,l]
    X_train, X_test, y_train, y_test = train_test_split(X_subset, y, test_size=0.20, random_state=123)

    X_train = np.reshape(X_train, (X_train.shape[0],X_train.shape[1],1))
    X_test = np.reshape(X_test, (X_test.shape[0],X_test.shape[1],1))
    model = Sequential()
    model.add(LSTM(11,return_sequences=True, input_shape = (X_train.shape[1],1)))
    model.add(layers.Flatten())
    model.add(Dense(1, kernel_initializer='lecun_uniform', activation='sigmoid'))
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=["accuracy"])
    model.fit(X_train, y_train,batch_size=64, verbose=1, workers=8, use_multiprocessing=True)
    P, test_acc = model.evaluate(X_test, y_test, verbose=0)

    particleScore.append(P)
    particleSize.append(X_subset.shape[1])

    j = (alpha * (1.0 - P)
        + (1.0 - alpha) * (1 - (X_subset.shape[1] / total_features)))
    

    return j
def f(x, alpha=0.9):

    n_particles = x.shape[0]
    j = [f_per_particle(x[i], alpha) for i in range(n_particles)]

    return np.array(j)

In [51]:
from datetime import datetime as dt
import time
from pyswarms.single import GlobalBestPSO
from pyswarms.discrete import BinaryPSO


start = dt.now()
print("Started at: ", str(start))

options = {'c1': 2, 'c2': 2, 'w':0.3, 'k': 1, 'p':2}


dimensions = X_r2.shape[1] 

optimizer =BinaryPSO(n_particles=2, dimensions=dimensions, options=options)

# Perform optimization
cost, pos = optimizer.optimize(f,iters=5, verbose=2)



2022-09-18 14:26:28,952 - pyswarms.discrete.binary - INFO - Optimize for 5 iters with {'c1': 2, 'c2': 2, 'w': 0.3, 'k': 1, 'p': 2}


Started at:  2022-09-18 14:26:28.945479


pyswarms.discrete.binary:   0%|          |0/5

7108/7108 [==============================] - 28s 4ms/step - loss: 0.1118 - accuracy: 0.9605


pyswarms.discrete.binary:  20%|██        |1/5, best_cost=0.871

...

pyswarms.discrete.binary:  60%|██████    |3/5, best_cost=0.871

7108/7108 [==============================] - 29s 4ms/step - loss: 0.0996 - accuracy: 0.9644


pyswarms.discrete.binary:  80%|████████  |4/5, best_cost=0.871

7108/7108 [==============================] - 29s 4ms/step - loss: 0.0944 - accuracy: 0.9652


pyswarms.discrete.binary: 100%|██████████|5/5, best_cost=0.871
2022-09-18 14:32:51,530 - pyswarms.discrete.binary - INFO - Optimization finished | best cost: 0.8709301592906317, best pos: [1 0 1 1 0 0 1 0 1 1 1 1 0 0 0 1 1 0 0 1 0 0 1 1 0 1 0 0 0 0]


In [ ]:
#history = model.fit(X_train, y_train, epochs=2, batch_size=32, verbose=1,validation_data=(X_test, y_test), callbacks=[EarlyStopper])

In [53]:
m=[1,0,1,1,0,0,1,0,1,1,1,1,0,0,0,1,1,0,0,1,0,0,1,1,0,1,0,0,0,0]
l=[]

for i in range(30):
    if m[i]==1:
        l.append(i)

X_r2[:,l]

array([[-1.82495518, -0.54803374,  1.00716849, ...,  0.24740559,
        -0.10405953,  0.16729189],
       [-1.82495518, -0.45440849,  0.62069626, ..., -0.54227882,
         0.1046054 ,  0.23045727],
       [-1.8249344 , -0.89813047,  0.88271786, ...,  0.67291038,
         0.9009109 , -0.57860701],
       ...,
       [-0.2281551 ,  0.00934472,  0.09589012, ..., -0.42025047,
        -0.39648085,  0.79648131],
       [ 0.47273592, -0.18801047, -0.07205724, ...,  0.20759798,
         0.26040556, -0.70547527],
       [ 1.34782899, -0.13845375,  0.64626979, ...,  0.66677198,
        -0.1540076 , -0.29874161]])

In [54]:
X_train, X_test, y_train, y_test = train_test_split(X_r2[:,l], y, test_size=0.20, random_state=None)
# Perform classification and store performance in P
X_train = np.reshape(X_train, (X_train.shape[0],X_train.shape[1],1))
X_test = np.reshape(X_test, (X_test.shape[0],X_test.shape[1],1))
model = Sequential()
model.add(LSTM(11,return_sequences=True, input_shape = (X_train.shape[1],1)))
model.add(layers.Flatten())
model.add(Dense(1, kernel_initializer='lecun_uniform', activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=["accuracy"])
model.fit(X_train, y_train,batch_size=64, verbose=1, workers=8, use_multiprocessing=True)
P, test_acc = model.evaluate(X_test, y_test, verbose=0)
print(f"Test Accuracy:{test_acc} Test Loss: {P}")

7108/7108 [==============================] - 30s 4ms/step - loss: 0.1451 - accuracy: 0.9465
Test Accuracy:0.9625855088233948 Test Loss: 0.0976981595158577


train_loss, train_acc = model.evaluate(X_train, y_train, verbose=0)
print(f"Train_Accuracy:{train_acc} Train Loss: {train_loss}")
test_loss, test_acc = model.evaluate(X_test, y_test, verbose=0)
print(f"Test Accuracy:{test_acc} Test Loss: {test_loss}")
y_pred = np.argmax(model.predict(X_test, verbose=0), axis=1)

'''
    if x[1]< 0.003:
        learning_rate = 0.001
    elif x[1]< 0.0075:
        learning_rate = 0.005
    elif x[1]< 0.015:
        learning_rate = 0.01
    elif x[1]< 0.035:
        learning_rate = 0.02
    elif x[1]< 0.075:
        learning_rate = 0.05
    elif x[1]< 0.125:
        learning_rate = 0.1
    elif x[1]< 0.175:
        learning_rate = 0.15
    else:
        learning_rate = 0.2
'''

from tensorflow.keras import backend as K

def model_design(x): 
  
    #print(f"Units : {int(x[0])}, Learning Rate : {x[2]:.2f}")
    K.clear_session()
    input_dim = X_train.shape[1]

    model = Sequential()
    model.add(LSTM(int(x[0]),return_sequences=True, input_shape = (X_train.shape[1], 1)))
    model.add(layers.Flatten())
    model.add(Dense(1, kernel_initializer='lecun_uniform', activation='sigmoid'))
    if x[1]< 0.003:
        learning_rate = 0.001
    elif x[1]< 0.0075:
        learning_rate = 0.005
    elif x[1]< 0.015:
        learning_rate = 0.01
    elif x[1]< 0.035:
        learning_rate = 0.02
    elif x[1]< 0.075:
        learning_rate = 0.05
    elif x[1]< 0.125:
        learning_rate = 0.1
    elif x[1]< 0.175:
        learning_rate = 0.15
    else:
        learning_rate = 0.2
    print(f"Units : {int(x[0])}, Learning Rate : {learning_rate:.2f}")
    
    opt = keras.optimizers.Adam(learning_rate=learning_rate)
    model.compile(loss='MeanSquaredError', optimizer=opt, metrics=["accuracy"])
    return model

EarlyStopper = EarlyStopping(patience=4, monitor='val_loss', mode='min')
count = 0
def best_model(x):
    print(x)
    model = model_design(x)

    history = model.fit(X_train, y_train, epochs=1, batch_size=64, verbose=1, validation_data=(X_test, y_test),callbacks=[EarlyStopper], workers=8, use_multiprocessing=True)

    train_loss, train_acc = model.evaluate(X_train, y_train, verbose=0)
    print(f"Train_Accuracy:{train_acc} Train Loss: {train_loss}")

    test_loss, test_acc = model.evaluate(X_test, y_test, verbose=0)
    print(f"Test Accuracy:{test_acc} Test Loss: {test_loss}")
    y_pred = np.argmax(model.predict(X_test, verbose=0), axis=1)
    #print(classification_report(y_test, y_pred))
    #matrix = confusion_matrix(y_test, y_pred)
    return test_loss


lb=[10,0.01]
ub=[25,0.02]
xopt, fopt = pso(best_model, lb, ub)
print ("Best position"+str(xopt))
print ("Loss:" + str(fopt))